![Shopping trolley in front of a laptop](./iStock-1249219777.jpg)

It's simple to buy any product with a click and have it delivered to your door. Online shopping has been rapidly evolving over the last few years, making our lives easier. But behind the scenes, e-commerce companies face a complex challenge that needs to be addressed. 

Uncertainty plays a big role in how the supply chains plan and organize their operations to ensure that the products are delivered on time. These uncertainties can lead to challenges such as stockouts, delayed deliveries, and increased operational costs.

You work for the Sales & Operations Planning (S&OP) team at a multinational e-commerce company. They need your help to assist in planning for the upcoming end-of-the-year sales. They want to use your insights to plan for promotional opportunities and manage their inventory. This effort is to ensure they have the right products in stock when needed and ensure their customers are satisfied with the prompt delivery to their doorstep.


## The Data

You are provided with a sales dataset to use. A summary and preview are provided below.

# Online Retail.csv

| Column     | Description              |
|------------|--------------------------|
| `'InvoiceNo'` | A 6-digit number uniquely assigned to each transaction |
| `'StockCode'` | A 5-digit number uniquely assigned to each distinct product |
| `'Description'` | The product name |
| `'Quantity'` | The quantity of each product (item) per transaction |
| `'UnitPrice'` | Product price per unit |
| `'CustomerID'` | A 5-digit number uniquely assigned to each customer |
| `'Country'` | The name of the country where each customer resides |
| `'InvoiceDate'` | The day and time when each transaction was generated `"MM/DD/YYYY"` |
| `'Year'` | The year when each transaction was generated |
| `'Month'` | The month when each transaction was generated |
| `'Week'` | The week when each transaction was generated (`1`-`52`) |
| `'Day'` | The day of the month when each transaction was generated (`1`-`31`) |
| `'DayOfWeek'` | The day of the weeke when each transaction was generated <br>(`0` = Monday, `6` = Sunday) |

In [31]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, dayofmonth, month, year,  to_date, to_timestamp, weekofyear, dayofweek
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize Spark session
my_spark = SparkSession.builder.appName("SalesForecast").getOrCreate()

# Importing sales data
sales_data = my_spark.read.csv(
    "Online Retail.csv", header=True, inferSchema=True, sep=",")

# Convert InvoiceDate to datetime 
sales_data = sales_data.withColumn("InvoiceDate", to_date(
    to_timestamp(col("InvoiceDate"), "d/M/yyyy H:mm")))

In [4]:
# Insert the code necessary to solve the assigned problems. Use as many code cells as you need.


In [32]:
sales_data = sales_data.withColumn("InvoiceDate", to_date(to_timestamp("InvoiceDate", "d/m/yyy H:mm")))

In [33]:
#using groupBy() to group all the data based on sum(quantity) and avg(unitprice)
daily_salaes_data = sales_data.groupBy(col("Country"),col("StockCode"),col("InvoiceDate"), col("Year"),col("Month"), col("Day"), col("Week"), col("DayOfWeek")).agg({"Quantity" : "sum", "UnitPrice" : "avg"})

In [34]:
#Renaming the Sum(Quantity) column:
daily_salaes_data = daily_salaes_data.withColumnRenamed("Sum(Quantity)", "Quantity")

In [35]:
#spliting the data into train and test:
#tain_data
train_data = daily_salaes_data.filter(col("InvoiceDate") <= "2011-09-25")

#test_data:
test_data = daily_salaes_data.filter(col("InvoiceDate") > "2011-09-25")

In [36]:
pd_daily_train_data = train_data.toPandas()

In [38]:
#creating the stringinderxer:
#country_indexe
country_indexe = StringIndexer(inputCol = "Country", outputCol= "CountryIndex").setHandleInvalid("keep")

stock_code_indexe = StringIndexer(inputCol = "StockCode", outputCol= "StockCodeIndex").setHandleInvalid("keep")

In [39]:
#defing the colums that will be used in ML
feature_cols = ["CountryIndex", "StockCodeIndex", "Month", "Year",
                "DayOfWeek", "Day", "Week"]

In [40]:
#VectorAssembeler: it combaines all the features into single vector feature
#this is done so because the ML models cannot take individual columns
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

In [41]:
#initialize a random forest model:
rf = RandomForestRegressor(featuresCol="features",
                          labelCol="Quantity",
                          maxBins=4000)

In [42]:
#create the pipeline for staging the processes:
pipeline = Pipeline(stages=[country_indexe,stock_code_indexe,assembler,rf])

In [43]:
#tring the model using trai_data:
model = pipeline.fit(train_data)

25/10/17 17:50:26 WARN DAGScheduler: Broadcasting large task binary with size 1623.6 KiB


25/10/17 17:50:28 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


In [44]:
#getting the predictins:
test_predictions = model.transform(test_data)
test_predictions = test_predictions.withColumn("prediction", col("prediction").cast("double"))


In [ ]:
# Provide the Mean Absolute Error (MAE) for your forecast? Return a double/floar "mae"

In [45]:
# Initializing the evaluator
mae_evaluator = RegressionEvaluator(labelCol="Quantity", predictionCol="prediction", metricName="mae")

In [46]:
# Obtaining MAE
mae = mae_evaluator.evaluate(test_predictions)

In [47]:
# How many units will be sold during the  week 39 of 2011? Return an integer `quantity_sold_w39`.

# Getting the weekly sales of all countries
weekly_test_predictions = test_predictions.groupBy("Year", "Week").agg({"prediction": "sum"})

In [48]:
# Finding the quantity sold on the 39 week. 
promotion_week = weekly_test_predictions.filter(col('Week')==39)

In [49]:
# Storing prediction as quantity_sold_w30
quantity_sold_w39 = int(promotion_week.select("sum(prediction)").collect()[0][0])

In [50]:
my_spark.stop()